In [ ]:
import numpy as np
import anndata as ad
import pandas as pd
import scanpy as sc
import random
import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pickle

np.random.seed(1024) 
random.seed(2048)
tf.random.set_seed(1024)

In [3]:
adata = sc.read_h5ad('data/sc_training.h5ad')
df = adata.obs[['condition', 'state']]
prop = df.groupby(by=['condition', 'state'],as_index=False).size()
total = df.groupby(by='condition',as_index=False).size()
prop = prop.merge(total, on='condition', how='left')
prop['prop'] = prop.size_x /prop.size_y
prop = prop.pivot_table(index='condition', columns='state', values='prop')
prop = prop[['progenitor', 'effector', 'terminal exhausted', 'cycling', 'other']]

state_wt = np.array(prop.loc['Unperturbed'].values)
test_set = ['Aqr', 'Bach2', 'Bhlhe40', 'Ets1', 'Fosb', 'Mafk', 'Stat3']

Define model

In [4]:
def regression_model(
    input_size = 32,
    output_size = 5,
    size_dense = 8,
    dropout_rate = 0.15,
    l1 = 1e-3,
    l2 = 1e-4,
    learning_rate = 1e-3,
    mae_weight=2
):
    def custom_KL_MAE_loss(y_true, y_pred):
        y_true = tf.multiply(y_true, [1,1,1,1,0.01])
        y_pred = tf.multiply(y_pred, [1,1,1,1,0.01])
        kl = tf.keras.metrics.kl_divergence(y_true, y_pred)
        mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
        return kl+mae*mae_weight
    
    input_layer = Input(shape=input_size)
    x = Dense(size_dense, activation='relu', kernel_regularizer=regularizers.L1L2(l1,l2))(input_layer)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(size_dense, activation='sigmoid', kernel_regularizer=regularizers.L1L2(l1,l2))(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    output_layer = Dense(output_size, activation='softmax')(x)

    model = Model(input_layer, output_layer)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss= custom_KL_MAE_loss, #"kullback_leibler_divergence",
                  metrics=['mae']
                 )
    return model


Prepare training dataset

In [5]:
emb_size = 32
g_embed = np.load(f'submission/embedding/gene_embedding_{emb_size}.npy')
g_name = np.load(f'submission/embedding/gene_names_{emb_size}.npy', allow_pickle= True)

genes = list(set(prop.index.tolist()))
training_conditions = [g for g in genes if g in g_name]

#extract embedding of training set
X, Y = [], []
for g in training_conditions:
    x = g_embed[list(g_name).index(g)]
    y = np.array(prop.loc[g].values)
    X.append(x)
    Y.append(y)

X = np.array(X)
Y = np.array(Y)

X_test = []
for g in test_set:
    x = g_embed[list(g_name).index(g)]
    X_test.append(x)
X_test = np.array(X_test)


train different model to map 32D gene embedding vector to 5D output vector

In [7]:
#scaler on the whole gene list
X_scaler = StandardScaler()
X_scaler.fit(g_embed)

with open('./submission/scaler.pkl', 'wb') as f:
    pickle.dump(X_scaler, f)

#transform test set
X_test_transformed = X_scaler.transform(X_test)

In [ ]:
seed=36
epochs=2000

X, Y = shuffle(X,Y,random_state=seed)
kf = KFold(n_splits=10)
kfold = kf.split(X, Y)

for n_try in [4,5,6,7,8]:

    score = []
    predictions = []

    for k, (train, val) in enumerate(kfold):
        X_train, Y_train = X[train], Y[train]
        X_val, Y_val = X[val], Y[val]

        X_train_transformed = X_scaler.transform(X_train)
        X_val_transformed = X_scaler.transform(X_val)

        for dropout_rate in [0]:
            for l1 in [1e-3, 1e-4,0]:
                for l2 in [0, 1e-5]:
                    for learning_rate in [1e-3]:

                        #define lr schedule
                        decay = learning_rate / (epochs-50) * 10                        
                        def scheduler(epoch, lr):
                            if epoch < 50:
                                return lr
                            else:
                                return lr * 1/(1+decay*epoch) #tf.math.exp(-0.1)

                        for mae_weight in [2]:
                            for batch_size in [8,16]:
                                model_path = f"./submission/checkpoints/NN/model_{emb_size}_{dropout_rate}_{l1}_{l2}_{learning_rate}_{mae_weight}_{batch_size}_{k}_{n_try}.h5"

                                #define model
                                model = regression_model(
                                            input_size = emb_size,
                                            output_size = 5,
                                            size_dense = 8,
                                            dropout_rate = dropout_rate,
                                            l1 = l1,
                                            l2 = l2,
                                            learning_rate = learning_rate,
                                            mae_weight=mae_weight
                                        )

                                #define callbacks
                                checkpoint = keras.callbacks.ModelCheckpoint(
                                    model_path,
                                    monitor='val_mae', 
                                    verbose=0, save_best_only=True, mode='min'
                                )
                                early_stop = keras.callbacks.EarlyStopping(monitor='val_mae', patience=200, mode='min')
                                schedule = keras.callbacks.LearningRateScheduler(scheduler, verbose=0)

                                # training
                                model.fit(X_train_transformed, Y_train, 
                                            # sample_weight=w, 
                                            epochs=epochs, 
                                            shuffle=True, 
                                            batch_size=batch_size,
                                            validation_data=(X_val_transformed,Y_val),
                                            callbacks = [checkpoint, early_stop, schedule],
                                            verbose = 0
                                            )

                                # calculate best val_mae
                                model.load_weights(model_path)
                                mae_train = mean_absolute_error(Y_train, model.predict(X_train_transformed))
                                mae_val = mean_absolute_error(Y_val, model.predict(X_val_transformed))

                                # predict test genes
                                Y_test_pred = model.predict(X_test_transformed)

                                score.append([
                                    n_try, k, dropout_rate, l1, l2, learning_rate, mae_weight, batch_size, mae_train, mae_val
                                ])
                                predictions.append(Y_test_pred)

                                print(n_try, k, dropout_rate, l1, l2, learning_rate, mae_weight, batch_size, mae_train, mae_val)

                                del model, model_path


    score = pd.DataFrame(
        score,
        columns = ["n_try", "k", "dropout_rate", "l1", "l2", "learning_rate", "mae_weight", "batch_size", "mae_train", "mae_val"]
    )
    score.to_csv(f"./submission/predictions/NN_emb32_score_try{n_try}.csv")

    predictions = np.array(predictions)
    np.save(f"./submission/predictions/NN_emb32_prediction_try{n_try}", predictions)